# **Speech Censor Bot**


---


A simple bot for censoring audio-visual clips. The notebook is inspired by the OpenVINO v2020.1 docs for [Offline Speech Recognition Demo](https://docs.openvinotoolkit.org/latest/_inference_engine_samples_speech_libs_and_demos_Offline_speech_recognition_demo.html) and [another](https://docs.openvinotoolkit.org/latest/_inference_engine_samples_speech_libs_and_demos_Speech_libs_and_demos.html) documentation on inference engine samples. 
The first link explains how the custom *KALDI* models can be used in the application. For demonstration purpose, we use the pre-trained model provided in the OpenVINO package itself.

Users can test/use the notebook to run custom files by editing the **TODO** sections in the code cells. 

**Note: The version for OpenVINO matters as it has been observed that different versions have slighly different information**

To understand the KALDI files, see [this](https://stackoverflow.com/questions/54428601/kaldis-objects-explained-in-laymans-term)

#**Section 1 : For Audio + Video**

##**Install packages and dependencies**

In [0]:
#For audio preprocessing and audio-video manipulation
import wave
!apt-get install libsox-fmt-all libsox-dev sox
!pip install ffmpeg

##**Install OpenVINO toolkit and dependencies**
We used a portable version of OpenVINO toolkit for LINUX-based systems, maintained and open-sourced by my friend [Muhammad Ali](https://github.com/alihussainia/OpenDevLibrary). Look into his repository for more insight into the below cell.

In [0]:
!wget "https://raw.githubusercontent.com/alihussainia/OpenDevLibrary/master/openvino_initialization_script.py"
!python openvino_initialization_script.py

## **Initialize the OpenVINO environment and download related files**
The bash script will
- Download pre-trained Intel Models
- Create configuration file (needed for making inference on speech)
- Required pre-requisites for LibriSpeech Model (graph file, etc)
- Test **Online** and **Offline** demos to validate if all pre-requisites were installed properly. 

**Note: If working on Google Collab, the online demo may not work and hence the execution may seem to get entrapped into a never ending cycle. To avoid this you must replace the "demo_speech_recognition.sh" file with a custom one.**

So, we tackle this issue by un-commenting the call below. This will replace the original file with a modified one, maintained in the repository.


In [0]:
#!wget -P "/content/" "https://raw.githubusercontent.com/PrashantDandriyal/Speech-Censor-Bot/master/Demo_Backup_Files/demo_speech_recognition.sh"
#!rm "/opt/intel/openvino_2020.1.023/deployment_tools/demo/demo_speech_recognition.sh" 
#!cp -f "/content/demo_speech_recognition.sh" "/opt/intel/openvino_2020.1.023/deployment_tools/demo/demo_speech_recognition.sh" 

In [0]:
!bash /opt/intel/openvino_2020.1.023/deployment_tools/demo/demo_speech_recognition.sh
# Note: Simply running the bash script using "!" gives permission issues. The issue is being tackled.

## **Running Offline DEMO**
*(To use it for custom WAV file, edit the "run_demo.sh" file and add the path to your file)*

The output generated by the *run_demo.sh* is similar to :


>[ INFO ] Using feature transformation /root/openvino_models/ir/intel/lspeech_s5_ext/FP32/lspeech_s5_ext.feature_transform        
[ INFO ] InferenceEngine API ver. 2.1 (build: 37988)        
[ INFO ] Device info:        
[ INFO ] 	CPU: MKLDNNPlugin ver. 2.1        
[ INFO ] Batch size: 8        
[ INFO ] Model loading time: 49.93 ms        
Recognition result:        
**HOW ARE YOU DOING**

We extract this output (in a naive way) by simply asking *sed* method to filter the console output as we wish to use only the text generated from the speech.

Next, we save this output to a txt file for the next step: forced alignment.



In [0]:
import os
# TODO:
#Add the path to your WAV file
!wget "https://raw.githubusercontent.com/PrashantDandriyal/Speech-Censor-Bot/master/negan.wav"
wav_path = "/content/negan.wav"

##**Preprocessing audio file**
As per the OpenVINO v2020.1 docs [here](https://docs.openvinotoolkit.org/latest/_inference_engine_samples_speech_libs_and_demos_Offline_speech_recognition_demo.html), WAV file needs to be in following format: RIFF WAVE PCM 16bit, 16kHz, 1 channel i.e.,

>Sample size : 16bit    
Sampling Rate : 16kHz    
Number of channels : 1        

We preprocess audio and convert it if needed and replace the old file with new.

In [0]:
def preprocess(org_aud_path):
  tx = wave.open(org_aud_path, 'r')
  print ("Initial Parameters:")
  !sox --i "$org_aud_path"
  if(tx.getnchannels() > 1):
    #Convert stereo to mono
    #and replace the original with new
    !sox "$org_aud_path" processed.wav channels 1
    !rm -r "$org_aud_path"
    !mv "processed.wav" "$org_aud_path"
    print("Converted Stereo to Mono")

  if(tx.getframerate() != 16000):
    #Downsample (if > 16k) and Upsample (if < 16k)
    #and replace the original with new
    !sox "$org_aud_path" processed.wav rate 16000
    !rm -r "$org_aud_path"
    !mv "processed.wav" "$org_aud_path"
    print("Changed sample rate to 16k")

    print("Processed file into the same path with name 'processed.wav' ")

preprocess(wav_path)
print("Update file parameters")
!sox --i "$wav_path"

The demo uses the "how_are_you_doing.wav" audio file stored in the location         
```/opt/intel/openvino/deployment_tools/demo/how_are_you_doing.wav```
This file is fed to the inference engine using the bash file ```run_demo.sh```. Instead of editing another bash file or creating a new one, we rename our WAV file to **how_are_you_doing.wav`** and replace the original file with ours.

In [0]:
%cd "/content/"

#Rename file here OR edit the bash file
!mv "$wav_path" "how_are_you_doing.wav"

#Replace the file for test on custom file by removing it first 
!rm -r "/opt/intel/openvino/deployment_tools/demo/how_are_you_doing.wav"
!cp "/content/how_are_you_doing.wav" "/opt/intel/openvino/deployment_tools/demo/"


##**Perform Inference**
The OpenVINO dependencies have successfully been installed and the environment has also been initialized. Its time to make the inference ! Run the cell to make inference. As the shell script echoes the result onto the terminal, we use ```sed``` piping to publish our results onto a text file. Another instance of the same command but without this pipe is run, to provide status of the inference.

In [0]:
!/opt/intel/openvino/data_processing/audio/speech_recognition/demos/offline_speech_recognition_demo/run_demo.sh 
#Running again to save the output
!/opt/intel/openvino/data_processing/audio/speech_recognition/demos/offline_speech_recognition_demo/run_demo.sh | sed '1,/Recognition result/d' > /content/out_text.txt 


##Important filepaths
####WAV file path: 
**"/opt/intel/openvino/deployment_tools/demo/how_are_you_doing.wav"**

####Configuration file path: 
**"/root/openvino_models/ir/intel/lspeech_s5_ext/FP32/speech_lib.cfg"**

##**Use *gentle* to obtain syncmap**
[Gentle](httpshttps://github.com/lowerquality/gentle://) is a forced aligner built on KALDI that automatically generates a synchronization map between a list of text fragments and an audio file containing the narration of the text. In computer science this task is known as forced alignment.

Due to some difficulties, we will be manually obtaining the syncmap files (a json + csv file) from the [site](https://lowerquality.com/gentle/).

In [0]:
#TODO: Add path to your json file
json_path = "/content/align.json"

##**Time to censor** 


1.   Get the list of profanity words from [here](https://raw.githubusercontent.com/PrashantDandriyal/Google-profanity-words/master/list.txt)
2.   Detect any such word and mute the uttrance using *ffmpeg*



In [0]:
%cd /content/
!wget "https://raw.githubusercontent.com/PrashantDandriyal/Google-profanity-words/master/list.txt"
profane_text = "/content/list.txt"

In [0]:
#Make a list out of all the words in the profane_text txt file. 
#This is to avoid repetitive file searching 
with open(profane_text) as f:
  cuss_list = [i.strip() for i in f]
f.close()

#TODO: Add any such word that was supposed to be detected as a profanic word
#The word "F*Ck" has been recognised as "For", so censoring it for the WAV file "negan.wav" 
cuss_list.append("for")

In [0]:
#Parsing the json content to a easily-accesible dictionary
import json
import pandas as pd

with open(json_path, 'r') as f:
    handle = json.load(f)

df = pd.DataFrame(columns=['word', 'start', 'end'])
rows_list = []

for i in handle['words']:
  #print(i)
  dict1 = {}
  if((i['word']).lower() in cuss_list):
    try:
      dict1.update({"word":i['word'], 
                "start":i['start'],
                "end":i['end']})   
      rows_list.append(dict1) 
    except KeyError:    #Sometimes the word is not properly detected and the entry is "'case': 'not-found-in-audio'"
      pass

df = pd.DataFrame(rows_list)
print(df.head())

In [0]:
#Traverse the entire transcript(text converted from speech) and look for any of such cuss word
#If any such word occurs, save its start and stop time (timestamp) t
test_file = open(text_path,"r")

dff = pd.DataFrame(columns=['swear', 'start', 'end'])
swear_dict_list = []

with open(text_path, 'r') as f:
    for line in f:  
      for word in line.split():
        if(word.upper() in cuss_list):    #Convert the word to Upper case and seach int cuss word list
          d = {}
          d.update({"swear":word, 
                "start":df.loc[word].start,
                "end":df.loc[word].end})
          swear_dict_list.append(d)
dff = pd.DataFrame(swear_dict_list)

As we have obtained the durations of all the profane words in the speech, we will now suppress them by muting/fading the slice of audio.

In [0]:
wav_path = "/opt/intel/openvino/deployment_tools/demo/how_are_you_doing.wav"
#Format of SOX command to fade in. Refer to(https://stackoverflow.com/questions/20127095/using-sox-to-change-the-volume-level-of-a-range-of-time-in-an-audio-file)
'''
fade [type] fade-in-length [stop-position(=) [fade-out-length]]

sox -m
    -t wav "|sox -V1 inputfile.wav -t wav - fade t 0 2.2 0.4" 
    -t wav "|sox -V1 inputfile.wav -t wav - trim 1.8 fade t 0.4 3.4 0.4 gain -6 pad 1.8"
    -t wav "|sox -V1 inputfile.wav -t wav - trim 4.8 fade t 0.4 0 0 pad 4.8"
    outputfile.wav gain 9.542
'''
for i in df.index: 
  start = df['start'][i]
  end = df['end'][i]
  duration = end-start
  #The duration of transition period when a fades in and fades out
  trans = 0.3
  fade_start = start-(trans/2)
  act_fade_start = start+(trans/2)
  #fade_start -> act_fade_start -> fade_end -> act_fade_end
  fade_end = end-(trans/2)
  act_fade_end = end+(trans/2)

  fade_duration = duration + trans
  #Note: Don't forget to add the $ before variable names
  print("For ",start,", ", end)
  !sox -m -t wav "|sox -V1 $wav_path -t wav - fade t 0 $act_fade_start $trans" -t wav "|sox -V1 $wav_path -t wav - trim $fade_start fade t $trans $fade_duration $trans gain -40 pad $fade_start" -t wav "|sox -V1 $wav_path -t wav - trim $fade_end fade t $trans 0 0 pad $fade_end" outputfile.wav gain 9.542
  #Replacing the input file with the outputfile
  !rm -r "$wav_path"
  !cp outputfile.wav "$wav_path"
  
  

In [0]:
#Copy the final censored audio file to working directory
!cp "$wav_path" "/content/final_audio.wav"

               


        
---
The above section marks an end to the audio censoring. To generated video outputs, get the link to the video file and proceed below.


#**Section 2 : Only for Video**
Replace the audio of the video with the censored audio by using ffmpeg

In [0]:
### TODO : Get the video file
!wget -P "/content/" "https://raw.githubusercontent.com/PrashantDandriyal/Speech-Censor-Bot/master/DocsResources/negan_clip_original.mp4"
vid_path = "/content/negan_clip_original.mp4"

In [0]:
!ffmpeg -i "$vid_path" -i "$wav_path" -c:v copy -map 0:v:0 -map 1:a:0 censored_vid.mp4
#The output is generated in the working directory with the name "censored_vid.mp4"